In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd

In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [8]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '12b', tickers, True)

In [ ]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
48,MNST,0.56,0.98,262.42,223.44,51.72,54.66,0.06,0.33,0.78,277.92,171.51,52.80,0.02,0.37,0.72,173.76,199.49,51.51,-0.00,9.466667
31,GTLB,1.82,0.96,433.94,760.90,59.83,67.42,0.13,1.04,0.66,300.62,402.13,76.59,0.28,1.08,0.75,550.21,621.78,53.83,-0.10,9.292308
85,WOLF,2.85,0.93,789.91,1088.30,7.04,14.77,1.10,1.81,0.67,597.81,1054.26,14.74,1.09,1.50,0.71,668.69,802.46,7.98,0.13,9.236364
24,ENPH,5.52,0.94,2240.94,2094.41,71.45,96.74,0.35,3.08,0.76,2432.32,2750.12,93.01,0.30,3.31,0.81,2465.19,2543.19,67.18,-0.06,9.228571
89,ZS,5.16,0.95,2187.01,2225.63,187.38,210.29,0.12,2.92,0.77,1939.50,2254.68,231.00,0.23,2.86,0.78,2675.37,1785.09,189.72,0.01,9.083333
6,BASE,0.65,0.95,218.36,245.40,15.44,17.07,0.11,0.43,0.70,129.96,238.83,16.89,0.09,0.35,0.79,268.73,293.34,14.09,-0.09,9.045455
28,FIVE,3.83,0.94,1204.90,1441.24,106.08,123.00,0.16,2.29,0.73,865.09,1587.01,115.88,0.09,2.27,0.68,1015.38,1341.12,103.76,-0.02,9.025000
44,LYFT,0.88,0.94,177.55,370.87,13.57,15.59,0.15,0.53,0.76,336.12,389.24,16.92,0.25,0.59,0.80,153.21,336.72,12.14,-0.11,9.000000
71,SQ,3.50,0.95,1015.42,1736.54,89.65,97.94,0.09,2.00,0.70,751.29,1290.12,100.27,0.12,2.51,0.79,1128.48,1553.43,81.38,-0.09,8.944444
84,UPWK,0.82,0.94,268.64,284.10,16.44,18.60,0.13,0.59,0.63,144.92,307.48,19.31,0.17,0.50,0.82,184.83,335.30,14.12,-0.14,8.938462


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '11b', tickers1, False)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
4,DAPP,0.31,0.98,164.09,182.75,15.38,11.98,-0.22,0.33,0.72,151.86,175.97,20.29,0.32,0.24,0.68,123.29,137.27,12.03,-0.22,9.709091
25,ONLN,0.72,0.95,214.38,368.61,45.07,55.01,0.22,0.54,0.66,92.97,193.49,48.83,0.08,0.64,0.66,107.72,247.95,39.79,-0.12,9.272727
51,XSW,1.47,0.96,943.96,499.65,193.08,175.72,-0.09,1.16,0.62,389.48,347.39,195.38,0.01,1.40,0.67,593.84,414.42,173.09,-0.10,9.155556
18,IWO,2.04,0.96,1224.80,1037.58,290.94,317.90,0.09,1.13,0.70,975.77,831.88,320.65,0.10,1.30,0.68,1029.87,904.20,282.40,-0.03,9.155556
2,ARKW,1.65,0.94,616.64,828.35,111.68,138.06,0.24,1.61,0.72,460.05,407.05,141.73,0.27,1.15,0.63,410.96,672.54,105.59,-0.05,9.150000
28,SMOG,1.13,0.96,528.52,733.53,100.67,109.00,0.08,0.96,0.68,347.54,500.39,108.23,0.08,0.72,0.72,386.55,679.94,100.55,-0.00,9.100000
0,ARKF,0.58,0.93,215.33,273.42,38.15,46.42,0.22,0.45,0.74,104.39,142.64,45.06,0.18,0.50,0.57,72.93,124.96,35.61,-0.07,8.981818
49,XME,0.62,0.94,345.01,210.03,56.69,62.19,0.10,0.39,0.67,320.80,189.18,60.25,0.06,0.51,0.74,417.01,299.33,52.92,-0.07,8.800000
11,ICLN,0.25,0.91,79.48,120.92,11.53,14.18,0.23,0.20,0.64,23.78,75.89,13.49,0.17,0.17,0.71,47.96,77.32,11.74,0.02,8.708696
10,FPX,0.90,0.93,423.44,525.60,122.05,135.38,0.11,0.71,0.65,202.76,236.19,137.13,0.12,0.73,0.68,368.98,307.09,117.59,-0.04,8.663636


In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, True)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
22,EVGO,0.29,0.96,124.34,117.45,4.20,6.04,0.44,0.45,0.74,103.50,119.47,7.00,0.67,0.14,0.74,62.63,110.54,3.94,-0.06,5.556150
6,APPS,1.17,0.93,250.31,578.59,1.86,4.56,1.45,1.28,0.61,184.09,610.81,3.27,0.76,0.78,0.70,222.48,508.96,1.56,-0.16,5.422312
14,BLNK,0.61,0.91,167.37,323.97,1.55,2.85,0.84,0.72,0.67,109.74,256.67,2.33,0.50,0.39,0.73,64.95,241.77,1.51,-0.03,5.245798
18,CRCT,0.37,0.91,81.13,160.05,5.75,7.62,0.32,0.49,0.62,36.38,153.86,6.88,0.20,0.25,0.70,73.97,148.37,5.00,-0.13,5.071691
35,LAZR,6.01,0.87,1099.25,2618.22,5.84,20.13,2.45,6.75,0.68,652.65,2207.47,12.35,1.11,4.08,0.73,1323.16,2868.78,5.83,-0.00,5.064586
24,GOGO,0.34,0.95,140.79,156.14,7.69,8.41,0.09,0.33,0.70,142.34,149.99,8.68,0.13,0.17,0.75,76.39,105.17,7.34,-0.05,5.032680
57,SABR,0.22,0.90,57.06,100.87,3.67,4.52,0.23,0.24,0.76,27.84,95.32,3.94,0.07,0.15,0.65,42.21,90.57,3.24,-0.12,4.859335
67,TRUP,2.48,0.89,647.71,738.20,49.73,63.71,0.28,2.88,0.78,879.78,1080.22,53.41,0.07,1.07,0.65,500.44,580.11,45.54,-0.08,4.842437
4,AMSSY,1.89,0.89,233.48,726.00,3.54,4.44,0.26,2.13,0.65,169.66,486.96,4.58,0.29,1.11,0.76,271.06,602.12,2.05,-0.42,4.812217
46,OPEN,0.36,0.88,72.40,134.51,1.67,2.20,0.32,0.52,0.69,68.82,102.54,2.24,0.34,0.21,0.71,48.77,132.12,1.55,-0.07,4.801471


In [ ]:
# read csv file
pred_other = pd.read_csv('results/pennystock_predictions_12b.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other = pred_other.merge(df3[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')



In [ ]:
pred_other.head()

,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain
0,AIRI,4.44,2025-01-24,4.09,4.84,4.66,AIRI,3.65,5.58,3.17,-0.18
1,AEXAF,0.00,2025-01-24,0.00,0.00,0.00,AEXAF,0.90,0.59,0.12,299.97
2,ALSMY,2.22,2025-01-24,2.14,2.32,2.40,ALSMY,2.06,2.44,2.13,-0.07
3,AMSC,25.06,2025-01-24,18.64,25.48,20.27,AMSC,18.58,30.70,15.83,-0.26
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26


In [ ]:
# Get all entries where predicted_close > current_price and Gain > 0

result = pred_other[(pred_other['predicted_close'] > pred_other['current_price']) & (pred_other['Gain'] > 0)]
# And largest based on 'Alloc
result = result.sort_values('Alloc', ascending=False)
result


,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
18,CRCT,5.75,2025-01-24,4.63,6.14,6.13,CRCT,7.62,6.88,5.00,0.32,5.071691
24,GOGO,7.69,2025-01-24,6.78,7.75,7.91,GOGO,8.41,8.68,7.34,0.09,5.032680
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26,4.812217
46,OPEN,1.67,2025-01-24,1.24,1.80,1.77,OPEN,2.20,2.24,1.55,0.32,4.801471
58,SAVE,0.40,2025-01-24,0.42,0.59,0.56,SAVE,1.42,1.29,0.07,2.56,4.753217
51,PLUG,2.38,2025-01-24,2.25,3.05,2.65,PLUG,4.34,2.93,1.99,0.82,4.746055
44,OMQS,0.17,2025-01-24,0.15,0.26,0.24,OMQS,0.39,0.44,0.15,1.24,4.745256
25,GRAB,4.81,2025-01-24,4.59,5.02,4.92,GRAB,5.44,7.41,3.95,0.13,4.660633
47,OTLY,0.64,2025-01-24,0.57,0.66,0.66,OTLY,1.15,1.04,0.51,0.80,4.455882
64,STEM,0.60,2025-01-24,0.51,0.65,0.62,STEM,1.75,1.03,0.50,1.94,4.410855


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '11b', tickers4, False)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
17,DIS,6.18,0.92,85.97,88.51,111.16,123.56,0.11,5.31,0.79,116.78,129.88,122.15,0.10,4.30,0.70,34.37,94.52,110.66,-0.00,5.844156
21,INTC,2.96,0.77,34.30,53.58,20.56,23.68,0.15,2.25,0.52,1.16,6.85,24.43,0.19,1.68,0.66,27.48,35.41,20.35,-0.01,3.966667
23,ISRG,21.82,0.67,316.25,-37.08,535.29,400.88,-0.25,15.17,0.70,217.32,38.30,528.18,-0.01,14.58,0.61,101.08,-80.34,409.23,-0.24,3.465714
6,BA,28.48,0.66,51.96,70.38,169.90,210.89,0.24,26.10,0.39,41.64,-91.07,219.51,0.29,16.75,0.50,-145.48,-51.17,171.50,0.01,3.297619
30,NVDA,4.21,0.67,90.03,-26.89,144.47,113.89,-0.21,5.15,0.55,95.94,-39.32,142.91,-0.01,2.14,0.31,19.48,-108.87,94.67,-0.34,3.214286
25,LLY,29.54,0.66,718.54,53.79,781.98,619.40,-0.21,24.60,0.64,833.52,-79.31,876.37,0.12,23.68,0.50,113.02,-204.12,458.52,-0.41,3.095238
16,DHR,17.28,0.71,71.54,73.87,234.84,207.73,-0.12,11.35,0.52,49.37,16.58,240.84,0.03,8.87,0.62,42.16,61.67,174.90,-0.26,2.654762
7,BKNG,222.62,0.64,2926.44,668.26,4903.85,4115.11,-0.16,174.72,0.74,3516.14,-708.25,5030.33,0.03,159.04,0.49,566.17,-981.64,3803.72,-0.22,2.321429
2,ADBE,35.16,0.67,44.73,296.96,430.57,482.72,0.12,23.96,0.58,-208.12,242.83,475.17,0.10,26.85,0.49,-135.56,161.81,393.86,-0.09,2.035714
5,AVGO,10.82,0.60,126.85,-166.28,232.55,198.19,-0.15,8.58,0.55,62.17,-161.03,225.41,-0.03,6.85,0.37,-17.14,-261.40,135.56,-0.42,1.619048


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '1a', tickers5, True)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
61,THQQF,1.15,0.62,1.99,5.95,2.75,3.45,0.25,1.52,0.43,-3.93,-0.03,4.28,0.55,0.85,0.54,0.89,1.72,2.93,0.06,1.388571
1,ADDDF,17.99,0.50,24.36,66.20,239.00,251.01,0.05,10.75,0.53,-46.01,-29.26,263.10,0.10,10.92,0.59,24.82,75.86,235.07,-0.02,0.000000
2,ADYEY,1.80,0.53,6.24,-6.80,14.78,15.40,0.04,1.15,0.36,-19.18,-4.64,16.87,0.14,1.06,0.58,9.10,4.96,14.27,-0.03,0.000000
3,BAK,1.24,0.33,-10.30,-1.91,3.78,4.45,0.18,1.07,0.43,-9.15,2.92,5.14,0.36,0.82,0.59,-4.37,12.86,4.00,0.06,0.000000
0,ASML,29.28,0.68,-85.94,522.02,714.36,726.59,0.02,25.88,0.51,43.25,143.26,790.66,0.11,18.20,0.62,561.13,560.81,663.80,-0.07,0.000000
5,BESIY,6.67,0.43,42.67,-52.17,137.69,142.03,0.03,4.66,0.33,-114.71,15.85,151.01,0.10,3.91,0.50,-5.09,-106.98,132.25,-0.04,0.000000
7,CDUAF,0.95,0.44,-0.22,1.34,24.15,24.44,0.01,0.72,0.52,4.18,4.70,26.12,0.08,0.62,0.64,3.78,12.92,22.20,-0.08,0.000000
8,COTY,0.89,0.61,3.14,3.29,6.81,7.25,0.06,0.64,0.50,-1.17,4.55,8.06,0.18,0.62,0.62,3.13,-0.55,6.70,-0.02,0.000000
9,COVTY,1.92,0.53,9.12,-3.99,28.65,30.17,0.05,1.08,0.52,-7.84,11.73,31.70,0.11,1.11,0.53,1.24,0.82,27.91,-0.03,0.000000
10,CDMGF,3.14,0.38,-9.33,22.99,22.23,22.97,0.03,2.19,0.21,-64.97,-15.58,23.73,0.07,1.96,0.17,-11.78,-9.40,22.89,0.03,0.000000


In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [10]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
23,VNM,0.63,0.88,50.95,80.05,11.17,12.34,0.10,0.39,0.69,22.26,60.16,12.31,0.10,0.41,0.64,27.93,71.59,11.53,0.03,7.600000
17,ILF,1.16,0.87,118.59,109.31,21.39,23.14,0.08,0.84,0.62,63.02,68.20,22.70,0.06,0.89,0.70,57.86,75.10,20.77,-0.03,7.075000
4,EPHE,1.27,0.85,107.98,124.74,24.25,25.98,0.07,0.85,0.56,11.61,39.18,26.44,0.09,0.90,0.64,48.21,84.93,24.23,-0.00,6.357143
7,EWI,1.50,0.83,165.86,132.17,36.60,33.61,-0.08,1.08,0.56,40.11,5.10,36.07,-0.01,0.94,0.58,64.02,22.61,32.72,-0.11,6.175000
15,EWZ,1.61,0.85,123.70,171.42,22.92,24.23,0.06,1.12,0.56,68.35,71.93,25.63,0.12,1.11,0.73,92.58,117.43,22.76,-0.01,6.000000
21,THD,2.77,0.77,170.04,253.84,57.22,64.46,0.13,1.90,0.53,15.19,56.10,70.36,0.23,2.04,0.65,55.82,135.17,58.26,0.02,5.930769
11,EWP,1.12,0.79,133.60,74.91,31.36,28.65,-0.09,0.89,0.72,110.54,56.40,31.25,-0.00,0.77,0.62,50.19,21.48,28.44,-0.09,5.566667
16,IDX,0.83,0.80,38.57,58.91,14.53,15.75,0.08,0.57,0.52,-4.17,19.54,16.00,0.10,0.72,0.56,7.01,27.13,14.78,0.02,5.500000
2,FLN,0.80,0.81,65.81,77.14,15.36,16.50,0.07,0.65,0.65,48.50,41.68,17.87,0.16,0.63,0.62,45.98,52.83,14.97,-0.03,5.385714
0,ARGT,3.92,0.63,398.44,3.14,87.44,72.62,-0.17,2.61,0.55,227.88,-58.55,73.87,-0.16,2.36,0.39,85.28,-261.55,64.27,-0.27,4.123529


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False

tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, True)

In [12]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
87,TEX,0.91,0.97,534.75,584.77,44.95,52.29,0.16,0.54,0.75,541.15,556.98,53.59,0.19,0.51,0.76,405.10,431.43,42.40,-0.06,9.512500
64,OLN,0.89,0.94,389.80,551.16,32.66,38.94,0.19,0.56,0.70,231.61,370.46,39.94,0.22,0.60,0.80,478.81,566.54,30.39,-0.07,9.084211
13,BGFV,0.38,0.92,47.42,144.10,1.73,2.32,0.35,0.25,0.62,23.79,97.64,2.24,0.30,0.20,0.80,66.84,141.12,1.35,-0.21,8.971429
37,F,0.23,0.95,129.00,138.72,9.71,10.53,0.08,0.13,0.74,84.26,127.94,11.91,0.23,0.10,0.74,80.41,107.42,8.74,-0.10,8.875000
66,PARA,0.77,0.90,116.67,294.79,10.65,14.91,0.40,0.41,0.61,127.69,224.77,12.28,0.15,0.46,0.79,175.44,288.91,9.43,-0.11,8.750000
3,AMR,5.58,0.91,2883.89,2144.78,189.68,237.56,0.25,3.14,0.81,3064.17,2751.07,233.85,0.23,4.15,0.70,2442.91,1069.35,191.10,0.01,8.740000
91,TWI,0.29,0.92,113.80,152.57,6.58,7.72,0.17,0.16,0.71,99.13,137.22,7.40,0.13,0.19,0.77,124.04,136.00,6.47,-0.02,8.729412
65,OSK,1.43,0.96,661.61,807.61,91.85,95.26,0.04,0.80,0.71,731.48,703.30,103.95,0.13,0.84,0.76,628.52,864.73,88.92,-0.03,8.600000
38,FCX,0.75,0.94,369.36,418.92,39.05,41.84,0.07,0.43,0.71,298.85,262.33,44.27,0.13,0.53,0.83,435.44,518.47,35.13,-0.10,8.542857
11,BAX,0.96,0.91,165.80,519.92,30.46,35.46,0.16,0.61,0.62,45.68,282.05,38.44,0.26,0.64,0.84,199.25,492.49,30.01,-0.01,8.537500


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
5,GLD,1.68,0.93,642.06,362.03,244.29,224.16,-0.08,1.57,0.74,659.87,229.95,246.26,0.01,1.36,0.68,425.83,231.82,225.11,-0.08,8.425000
8,SLV,0.29,0.95,132.23,107.73,27.75,26.56,-0.04,0.24,0.72,136.39,79.96,29.20,0.05,0.25,0.76,104.09,121.44,26.99,-0.03,8.250000
12,USO,1.26,0.90,432.43,293.03,74.21,68.29,-0.08,0.89,0.72,458.09,424.87,77.31,0.04,1.04,0.79,580.40,395.94,72.13,-0.03,7.750000
4,FXY,0.55,0.90,92.01,206.92,60.05,64.12,0.07,0.45,0.48,-3.23,95.29,63.79,0.06,0.49,0.72,40.69,128.50,60.41,0.01,7.571429
15,WEAT,0.14,0.87,28.40,49.00,4.82,5.21,0.08,0.11,0.50,23.24,35.11,5.56,0.15,0.09,0.76,34.96,38.66,4.99,0.03,7.075000
13,UUP,0.13,0.91,48.33,28.86,30.26,29.08,-0.04,0.10,0.72,49.66,23.11,30.11,-0.01,0.10,0.75,42.37,17.48,28.82,-0.05,6.850000
0,DBC,0.25,0.94,84.85,69.56,22.35,21.91,-0.02,0.17,0.72,92.84,70.64,23.19,0.04,0.20,0.69,93.23,49.20,22.72,0.02,6.400000
11,TLT,1.05,0.90,258.69,398.60,90.15,92.91,0.03,0.82,0.64,177.46,285.62,94.83,0.05,0.82,0.81,229.81,466.83,89.79,-0.00,5.666667
6,ISHG,0.39,0.90,97.21,141.01,70.10,71.60,0.02,0.26,0.64,38.69,62.08,73.17,0.04,0.27,0.80,81.98,128.39,70.07,-0.00,4.000000
9,SOYB,0.28,0.94,90.36,83.15,21.09,21.35,0.01,0.16,0.80,117.62,80.57,21.81,0.03,0.19,0.75,58.01,92.69,20.93,-0.01,3.400000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
7,FIL-USD,0.40,0.88,79.45,82.02,5.06,6.55,0.29,0.40,0.72,96.35,100.40,7.27,0.44,0.21,0.66,35.05,16.53,5.15,0.02,5.871921
11,MATIC-USD,0.05,0.83,10.13,7.97,0.48,1.10,1.29,0.04,0.65,8.23,9.33,0.81,0.69,0.02,0.70,6.45,7.42,0.44,-0.09,5.796788
12,NEAR-USD,0.32,0.83,78.82,34.34,5.15,7.10,0.38,0.29,0.61,53.99,55.12,7.78,0.51,0.15,0.67,45.50,41.58,4.75,-0.08,5.481203
3,BCH-USD,16.20,0.79,3011.30,1646.42,458.62,798.58,0.74,16.04,0.68,4677.31,3021.20,637.30,0.39,5.22,0.74,3282.53,1629.87,488.43,0.06,5.359073
14,TON-USD,0.11,0.85,20.24,13.02,1.73,2.06,0.19,0.11,0.71,66.46,67.45,2.19,0.26,0.05,0.80,19.30,24.76,1.77,0.02,5.281955
1,AVAX-USD,1.79,0.86,505.66,310.42,38.54,44.85,0.16,1.97,0.71,374.40,280.93,51.25,0.33,0.99,0.70,256.45,147.97,38.75,0.01,5.267857
0,ADA-USD,0.04,0.84,3.94,2.31,0.92,0.74,-0.19,0.04,0.65,4.78,2.12,0.89,-0.04,0.02,0.76,4.11,3.91,0.76,-0.18,5.157895
5,DOT-USD,0.48,0.79,63.53,62.26,7.12,9.66,0.36,0.48,0.70,60.57,58.33,9.37,0.32,0.21,0.80,47.98,73.84,7.26,0.02,5.059524
8,ICP-USD,0.56,0.84,113.80,79.72,10.43,12.21,0.17,0.58,0.56,116.58,74.72,14.28,0.37,0.22,0.66,76.16,26.77,10.41,-0.00,5.058824
2,BTC-USD,2858.34,0.82,385894.94,175435.81,97518.11,79838.03,-0.18,2726.69,0.79,437012.33,67909.88,100868.93,0.03,1405.71,0.58,262614.15,-91883.34,79487.05,-0.18,4.857143


In [ ]:
# read csv file
pred_other = pd.read_csv('results/bluechipstock_predictions_1a.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other = pred_other.merge(df4[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')


In [ ]:

result = pred_other
# And largest based on 'Alloc
result = result.sort_values('Alloc', ascending=False)
result

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
17,DIS,111.16,2025-01-03,2025-01-31,110.08,115.17,112.30,DIS,123.56,122.15,110.66,0.11,5.844156
21,INTC,20.56,2025-01-03,2025-01-31,20.18,21.50,21.10,INTC,23.68,24.43,20.35,0.15,3.966667
23,ISRG,535.29,2025-01-03,2025-01-31,516.35,581.27,533.40,ISRG,400.88,528.18,409.23,-0.25,3.465714
6,BA,169.90,2025-01-03,2025-01-31,165.89,190.41,182.76,BA,210.89,219.51,171.50,0.24,3.297619
30,NVDA,144.47,2025-01-03,2025-01-31,138.57,149.89,143.06,NVDA,113.89,142.91,94.67,-0.21,3.214286
25,LLY,781.98,2025-01-03,2025-01-31,750.62,822.27,797.46,LLY,619.40,876.37,458.52,-0.21,3.095238
16,DHR,234.84,2025-01-03,2025-01-31,228.52,247.12,238.90,DHR,207.73,240.84,174.90,-0.12,2.654762
7,BKNG,4903.85,2025-01-03,2025-01-31,4851.59,5090.64,4921.39,BKNG,4115.11,5030.33,3803.72,-0.16,2.321429
2,ADBE,430.57,2025-01-03,2025-01-31,420.70,447.15,430.98,ADBE,482.72,475.17,393.86,0.12,2.035714
5,AVGO,232.55,2025-01-03,2025-01-31,225.36,242.40,241.31,AVGO,198.19,225.41,135.56,-0.15,1.619048


In [ ]:
# read csv file
pred_other1 = pd.read_csv('results/global_predictions_1a.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other1 = pred_other1.merge(df5[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')


In [ ]:

result1 = pred_other1[(pred_other1['predicted_close'] > pred_other1['current_price']) & (pred_other1['Gain'] > 0)]
# And largest based on 'Alloc
result1 = result1.sort_values('Alloc', ascending=False)
result1

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
61,THQQF,2.76,2025-01-03,2025-01-31,2.65,2.82,2.78,THQQF,3.45,4.28,2.93,0.25,1.388571
0,ASML,714.36,2025-01-03,2025-01-31,691.26,789.68,757.36,ASML,726.59,790.66,663.80,0.02,0.000000
2,ADYEY,14.78,2025-01-03,2025-01-31,14.16,15.47,15.05,ADYEY,15.40,16.87,14.27,0.04,0.000000
1,ADDDF,239.00,2025-01-03,2025-01-31,229.15,253.52,244.28,ADDDF,251.01,263.10,235.07,0.05,0.000000
3,BAK,3.78,2025-01-03,2025-01-31,3.66,4.16,3.99,BAK,4.45,5.14,4.00,0.18,0.000000
4,BASFY,10.66,2025-01-03,2025-01-31,10.30,11.54,11.00,BASFY,11.01,11.67,10.43,0.03,0.000000
8,COTY,6.81,2025-01-03,2025-01-31,6.58,7.51,7.21,COTY,7.25,8.06,6.70,0.06,0.000000
5,BESIY,137.69,2025-01-03,2025-01-31,132.71,148.89,143.05,BESIY,142.03,151.01,132.25,0.03,0.000000
13,DNNGY,15.97,2025-01-03,2025-01-31,15.25,16.98,16.03,DNNGY,17.36,19.44,15.36,0.09,0.000000
16,EDNMF,36.35,2025-01-03,2025-01-31,33.96,36.22,37.41,EDNMF,37.23,38.80,34.70,0.02,0.000000


In [13]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
etf-12a-prediction.csv
val-1a-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
val-1a-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
val-1a-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
val-1a-prediction.csv

sent 28,845,278 bytes  received 111 bytes  57,690,778.00 bytes/sec
total size is 57,658,739  speedup is 2.00
